In [1]:
import pandas as pd

# Входные параметры
TARGET_GENE = "SIK3"
TARGET_CANCER_TYPE = "Cervical Cancer"

# Загрузка данных
# 1. Информация о типах рака (включая StrippedCellLineName)
cancer_types = pd.read_csv(r"../../../depmap_datasets/prepared_datasets/cancer_cell_lines_merged_data.csv")
# 2. LFC PIM-447
lfc_data = pd.read_csv(r"../../../depmap_datasets/prepared_datasets/lfc_pim-447_prism.csv")
# 3. Депскоры генов
depscores = pd.read_csv(r"../../../depmap_datasets/prepared_datasets/depscores.csv")

# Фильтрация по типу рака + выбор нужных столбцов (включая StrippedCellLineName)
filtered_lines = cancer_types[cancer_types["primary_disease_renamed"] == TARGET_CANCER_TYPE][
    ["ModelID", "primary_disease_renamed", "StrippedCellLineName"]
]

# Выбираем только нужный ген из depscores
depscores_filtered = depscores[["DepMap_ID", TARGET_GENE]].rename(columns={"DepMap_ID": "ModelID"})

# Объединяем данные
merged_data = (
    filtered_lines
    .merge(depscores_filtered, on="ModelID", how="inner")
    .merge(lfc_data[["ModelID", "LFC"]], on="ModelID", how="inner")
)

# Заполняем пропуски в StrippedCellLineName значением ModelID
merged_data["StrippedCellLineName"] = merged_data["StrippedCellLineName"].fillna(merged_data["ModelID"])

# Переименовываем столбцы для читаемости
merged_data.columns = ["ModelID", "Cancer_Type", "CellLine_Name", TARGET_GENE, "PIM-447_LFC"]

# Обработка дубликатов (оставляем строку с минимальным LFC)
print("\nПроверка дубликатов:")
duplicates = merged_data[merged_data.duplicated(subset=["ModelID"], keep=False)]
if not duplicates.empty:
    print("Найдены дубликаты (несколько экспериментов для одной линии):")
    print(duplicates.sort_values(by="ModelID"))
    
    # Оставляем только строку с минимальным LFC
    merged_data = merged_data.loc[merged_data.groupby("ModelID")["PIM-447_LFC"].idxmin()]
    print("\nОставлены строки с минимальным LFC:")
    print(merged_data[merged_data["ModelID"].isin(duplicates["ModelID"])])
else:
    print("Дубликатов не найдено.")

# Упорядочиваем колонки (CellLine_Name будет второй колонкой)
merged_data = merged_data[["ModelID", TARGET_GENE, "PIM-447_LFC", "CellLine_Name", "Cancer_Type"]]

# Сохраняем в CSV
output_filename = f"{TARGET_CANCER_TYPE.replace(' ', '_').replace('/', '_').lower()}_{TARGET_GENE.lower()}_pim-447_data.csv"
merged_data.to_csv(output_filename, index=False)

print(f"\nДанные сохранены в файл: {output_filename}")
print(f"Найдено {len(merged_data)} клеточных линий для {TARGET_CANCER_TYPE} и гена {TARGET_GENE}")


Проверка дубликатов:
Дубликатов не найдено.

Данные сохранены в файл: cervical_cancer_sik3_pim-447_data.csv
Найдено 13 клеточных линий для Cervical Cancer и гена SIK3
